In [1]:
import numpy as np
import pandas as pd
import os
import argparse
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import time

In [2]:
from keras.models import Sequential, Model
from keras.layers import Input,Embedding,LSTM,Dense
from keras.optimizers import Adam
from keras.layers import Input, concatenate, Embedding, Reshape
from keras.layers import Merge, Flatten, merge, Lambda, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2, l1_l2
import tensorflow as tf

/Users/BharathiSrinivasan/anaconda2/envs/python36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
myfolder = '/Users/BharathiSrinivasan/Documents/HU-MEMS-Sem3/Info_Systems/repo/Recommender_DNN/Data/'
big_users = pd.read_csv(myfolder + 'bigtrain.csv')
#test = pd.read_csv(myfolder + 'bigtest.csv')
products = pd.read_csv(myfolder + "products.csv")

#To Do : Include this in pre-processing dataset while creating bigtrain and bigtest
#train.dropna()
#test.dropna()

In [4]:
df = big_users

#Only keep orders which have been reordered
df_seq = df[df.reordered==1]

#To Do : Include this in pre-processing dataset while creating bigtrain and bigtest
df.drop(['eval_set'],axis=1)

,user_id,user_orders,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id
0,17,40,1737705,1,2,13,30,7350,1,0,115,7
1,17,40,1681401,2,5,10,3,7350,1,1,115,7
2,17,40,2680214,3,3,10,5,7350,1,1,115,7
3,17,40,3237467,5,6,17,5,7350,1,1,115,7
4,17,40,2616505,6,4,17,5,7350,2,1,115,7
5,17,40,2648319,7,1,13,4,7350,1,1,115,7
6,17,40,2430354,8,0,15,6,7350,1,1,115,7
7,17,40,900554,10,0,15,4,7350,1,1,115,7
8,17,40,805025,11,4,15,4,7350,2,1,115,7
9,17,40,912404,12,2,14,5,7350,1,1,115,7


In [19]:
#Merge with products dataset
df_seq = pd.merge(df_seq,products,on='product_id',how='left')
df_seq[['user_id','order_id','product_id','product_name','reordered']].head()

,user_id,order_id,product_id,product_name,reordered
0,17,1681401,7350,Natural Lime Flavor Sparkling Mineral Water,1
1,17,2680214,7350,Natural Lime Flavor Sparkling Mineral Water,1
2,17,3237467,7350,Natural Lime Flavor Sparkling Mineral Water,1
3,17,2616505,7350,Natural Lime Flavor Sparkling Mineral Water,1
4,17,2648319,7350,Natural Lime Flavor Sparkling Mineral Water,1


In [20]:
#Creating baskets of product sequences for each user ID
df_seq["product_name"] = df_seq["product_name"].astype(str)
df_seq = df_seq.groupby("user_id").apply(lambda order: order['product_name'].tolist())
df_seq = df_seq.reset_index()
df_seq.columns = ['user_id','product_set']
df_seq.product_set = df_seq.product_set.astype(str)
df_seq.head()

,user_id,product_set
0,17,['Natural Lime Flavor Sparkling Mineral Water'...
1,21,"['Hard Boiled Eggs', 'Hard Boiled Eggs', 'Hard..."
2,27,['Total 2% All Natural Greek Strained Yogurt w...
3,36,"['Sparkling Water Grapefruit', 'Sparkling Wate..."
4,50,"['Organic Strawberries', 'Organic Strawberries..."


In [5]:
#PRE-PROCESSING
y = df['reordered'].values
df.pop('reordered')

0           0
1           1
2           1
3           1
4           1
5           1
6           1
7           1
8           1
9           1
10          1
11          1
12          1
13          1
14          1
15          1
16          1
17          1
18          1
19          1
20          1
21          1
22          1
23          1
24          1
25          1
26          1
27          1
28          1
29          1
           ..
16587477    1
16587478    1
16587479    1
16587480    1
16587481    1
16587482    1
16587483    1
16587484    0
16587485    1
16587486    1
16587487    0
16587488    0
16587489    0
16587490    0
16587491    0
16587492    0
16587493    0
16587494    0
16587495    0
16587496    1
16587497    0
16587498    0
16587499    1
16587500    0
16587501    0
16587502    0
16587503    0
16587504    0
16587505    0
16587506    0
Name: reordered, Length: 16587507, dtype: int64

In [6]:
CATEGORICAL_COLUMNS = ["order_dow", "order_hour_of_day"]
CONTINUOUS_COLUMNS = [ "user_orders", "order_number", "add_to_cart_order","days_since_prior_order"]

#One-hot encoding categorical columns
df = pd.get_dummies(df, columns=[x for x in CATEGORICAL_COLUMNS])

In [7]:
del(df['eval_set'])
del(df['aisle_id'])
del(df['department_id'])
df.head()

,user_id,user_orders,order_id,order_number,days_since_prior_order,product_id,add_to_cart_order,order_dow_0,order_dow_1,order_dow_2,...,order_hour_of_day_14,order_hour_of_day_15,order_hour_of_day_16,order_hour_of_day_17,order_hour_of_day_18,order_hour_of_day_19,order_hour_of_day_20,order_hour_of_day_21,order_hour_of_day_22,order_hour_of_day_23
0,17,40,1737705,1,30,7350,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,17,40,1681401,2,3,7350,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17,40,2680214,3,5,7350,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17,40,3237467,5,5,7350,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,17,40,2616505,6,5,7350,2,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [8]:
#To-do: Include in pre-processing
df[df==np.inf]=np.nan
df.dropna()
np.all(np.isfinite(df))
#df[np.isfinite(df) | np.isnan(df)]

True

In [9]:
df.reset_index()
#To create the numeric feature vectors
df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)
X = df.values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
# simply connecting the features to an output layer
wide_inp = Input(shape=(X_train.shape[1],), dtype='float32', name='wide_inp')
w = Dense(1, activation='sigmoid')(wide_inp)
wide = Model(wide_inp, w)
wide.compile(Adam(0.01), loss='binary_crossentropy', metrics=['accuracy'])
wide.fit(X_train, y_train, nb_epoch=1,batch_size =64)
results = wide.evaluate(X_test, y_test)

print("\n", results)

/Users/BharathiSrinivasan/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/1
4146877/4146877 [==============================] - 79s 19us/step

 [0.565779446195662, 0.734914973364092]


In [14]:
def val2idx(df, cols):
    """helper to index categorical columns before embeddings.
    """
    val_types = dict()
    for c in cols:
        val_types[c] = df[c].unique()

    val_to_idx = dict()
    for k, v in val_types.items():
        val_to_idx[k] = {o: i for i, o in enumerate(val_types[k])}

    for k, v in val_to_idx.items():
        df[k] = df[k].apply(lambda x: v[x])

    unique_vals = dict()
    for c in cols:
        unique_vals[c] = df[c].nunique()

    return df, unique_vals

In [20]:
#Creating embeddings for user_ID, order_ID and product_ID
EMBEDDING_COLUMNS = ["user_id", "product_id","order_id"]

df_deep, unique_vals = val2idx(df, EMBEDDING_COLUMNS)
X_deep_tr, X_deep_te, y_deep_tr, y_deep_te = train_test_split(df_deep, y, test_size=0.25, random_state=42)

def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

def continous_input(name):
    inp = Input(shape=(1,), dtype='float32', name=name)
    return inp, Reshape((1, 1))(inp)

DEEP_COLNS = EMBEDDING_COLUMNS + CONTINUOUS_COLUMNS

embeddings_tensors = []
n_factors = 8
reg = 1e-3
for ec in EMBEDDING_COLUMNS:
    layer_name = ec + '_inp'
    t_inp, t_build = embedding_input(
    layer_name, unique_vals[ec], n_factors, reg)
    embeddings_tensors.append((t_inp, t_build))
    del(t_inp, t_build)
    
continuous_tensors = []
for cc in CONTINUOUS_COLUMNS:
    layer_name = cc + '_in'
    t_inp, t_build = continous_input(layer_name)
    continuous_tensors.append((t_inp, t_build))
    del(t_inp, t_build)
    
X_train_deep = [X_deep_tr[c] for c in DEEP_COLNS]
y_train_deep = np.array(y_deep_tr).reshape(-1, 1)
X_test_deep = [X_deep_te[c] for c in DEEP_COLNS]
y_test_deep = np.array(y_deep_te).reshape(-1, 1)

In [22]:
#Building inputs for deep network
inp_layer =  [et[0] for et in embeddings_tensors]
inp_layer += [ct[0] for ct in continuous_tensors]
inp_embed =  [et[1] for et in embeddings_tensors]
inp_embed += [ct[1] for ct in continuous_tensors]

#Modeling deep network

d = merge(inp_embed, mode='concat')
d = Flatten()(d)
# 2_. layer to normalise continous columns with the embeddings
d = BatchNormalization()(d)
d = Dense(100, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(d)
d = Dense(50, activation='relu')(d)

deep_out = Dense(y_train_deep.shape[1], activation='sigmoid')(d)
deep = Model(inp_layer, deep_out)
deep.compile(Adam(0.01), loss='binary_crossentropy', metrics=['accuracy'])
deep.fit(X_train_deep, y_train_deep, batch_size=64, nb_epoch=1)
results = deep.evaluate(X_test_deep, y_test_deep)

print ("\n", results)

/Users/BharathiSrinivasan/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if __name__ == '__main__':
/Users/BharathiSrinivasan/anaconda2/envs/python36/lib/python3.6/site-packages/keras/legacy/layers.py:464: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/BharathiSrinivasan/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/1
  748224/12440630 [>.............................] - ETA: 12:37:11 - loss: 0.8052 - acc: 0.7425

KeyboardInterrupt: 

In [ ]:
#Wide + Deep

X_tr_wd = [X_train] + X_train_deep
Y_tr_wd = y_train_deep  # wide or deep is the same here
X_te_wd = [X_test] + X_test_deep
Y_te_wd = y_test_deep  # wide or deep is the same here

# WIDE
w = Input(shape=(X_train.shape[1],), dtype='float32', name='wide')

# WIDE + DEEP
wd_inp = concatenate([w, d])
wd_out = Dense(Y_tr_wd.shape[1], activation='sigmoid', name='wide_deep')(wd_inp)
wide_deep = Model(inputs=[w] + inp_layer, outputs=wd_out)
wide_deep.compile(optimizer=Adam(lr=0.01), loss='binary_crossentropy', metrics=['accuracy'])
wide_deep.fit(X_tr_wd, Y_tr_wd, nb_epoch=1, batch_size=128)

# Maybe you want to schedule a second search with lower learning rate
# wide_deep.optimizer.lr = 0.0001
# wide_deep.fit(X_tr_wd, Y_tr_wd, nb_epoch=10, batch_size=128)

results = wide_deep.evaluate(X_te_wd, Y_te_wd)

print( "\n", results)

/Users/BharathiSrinivasan/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  app.launch_new_instance()


Epoch 1/1
  268160/12440630 [..............................] - ETA: 9:21:56 - loss: 0.7432 - acc: 0.7464

In [60]:
# serialize model to JSON
wide_deep2_json = wide_deep.to_json()
with open("wide_deep2.json", "w") as json_file:
    json_file.write(wide_deep2_json)
# serialize weights to HDF5
model.save_weights("wide_deep2.h5")
print("Saved model to disk")

Saved model to disk


In [62]:
""""
#Later
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
"""""

'"\n#Later\n# load json and create model\njson_file = open(\'model.json\', \'r\')\nloaded_model_json = json_file.read()\njson_file.close()\nloaded_model = model_from_json(loaded_model_json)\n# load weights into new model\nloaded_model.load_weights("model.h5")\nprint("Loaded model from disk")\n'